In [ ]:
# AutoScale Pipeline - single-file runnable in Colab
# Features:
# - Mount Google Drive (optional)
# - Collect images from a folder
# - Heuristic auto-labeling (EasyOCR + contour expansion) when labels missing
# - Create YOLOv8 dataset structure and split
# - Train YOLOv8 (yolov8n) and save best weights
# - Run inference on test set and save annotated images + CSV of confidences


import os
import sys
import shutil
import random
import glob
import csv
from pathlib import Path
import cv2
import numpy as np

# --------- Configuration ---------
SOURCE_DIR = "/content"  # change to your path (after mounting Drive if needed)
OUTPUT_DATASET = "/content/dataset"
AUTO_LABEL_DIR = "/content/auto_labels"
IMAGE_EXTS = (".jpg", ".jpeg", ".png")
CLASS_NAME = "weighing_scale"
SEED = 42
SPLIT = {"train": 0.7, "val": 0.2, "test": 0.1}
YOLO_EPOCHS = 25
IMG_SIZE = 640
BATCH = 8

# --------- Helpers ---------
def pip_install(pkgs: list):
    import subprocess
    for p in pkgs:
        subprocess.run([sys.executable, "-m", "pip", "install", p], check=True)

# Install required packages if missing
try:
    import easyocr
except Exception:
    pip_install(["easyocr","ultralytics","opencv-python","numpy"])
    import easyocr
    from ultralytics import YOLO

from ultralytics import YOLO
import easyocr
import torch

# Create directories
os.makedirs(AUTO_LABEL_DIR, exist_ok=True)
os.makedirs(OUTPUT_DATASET, exist_ok=True)

# --------- Auto-labeling Routine (OCR + Contour heuristic) ---------
# Fixed: Using torch.cuda.is_available() for more reliable GPU check in Colab
reader = easyocr.Reader(["en"], gpu=torch.cuda.is_available())

def find_display_box_by_ocr(img):
    # returns list of boxes in xyxy (x1,y1,x2,y2)
    h, w = img.shape[:2]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # apply OCR
    try:
        results = reader.readtext(img)
    except Exception:
        results = []
    boxes = []
    for (bbox, text, conf) in results:
        if len(text.strip()) == 0:
            continue
        if any(ch.isdigit() for ch in text):
            xs = [int(max(0, min(w-1, p[0]))) for p in bbox]
            ys = [int(max(0, min(h-1, p[1]))) for p in bbox]
            x1, x2 = min(xs), max(xs)
            y1, y2 = min(ys), max(ys)
            pad_x = int((x2-x1) * 0.6)
            pad_y = int((y2-y1) * 0.6)
            x1 = max(0, x1 - pad_x)
            x2 = min(w-1, x2 + pad_x)
            y1 = max(0, y1 - pad_y)
            y2 = min(h-1, y2 + pad_y)
            boxes.append((x1,y1,x2,y2))
    return boxes

def find_display_box_by_contour(img):
    h, w = img.shape[:2]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edged = cv2.Canny(blur, 50, 150)
    cnts, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    candidates = []
    for c in cnts:
        area = cv2.contourArea(c)
        if area < (w*h)*0.001:
            continue
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02*peri, True)
        x,y,ww,hh = cv2.boundingRect(approx)
        aspect = ww/float(hh+1e-6)
        if 0.6 < aspect < 4.0:
            candidates.append((area, x,y,x+ww,y+hh))
    if not candidates:
        return []
    candidates.sort(reverse=True, key=lambda x: x[0])
    _, x1,y1,x2,y2 = candidates[0]
    pad_x = int((x2-x1)*0.25)
    pad_y = int((y2-y1)*0.25)
    x1 = max(0, x1-pad_x); x2=min(w-1, x2+pad_x)
    y1 = max(0, y1-pad_y); y2=min(h-1, y2+pad_y)
    return [(x1,y1,x2,y2)]

def img_to_yolo_label(img_path, boxes, out_label_path):
    img = cv2.imread(img_path)
    if img is None: return
    h,w = img.shape[:2]
    with open(out_label_path, "w") as f:
        for (x1,y1,x2,y2) in boxes:
            xc = ((x1 + x2)/2.0) / w
            yc = ((y1 + y2)/2.0) / h
            bw = (x2 - x1) / w
            bh = (y2 - y1) / h
            f.write(f"0 {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")

# Process images: create labels if missing
images = [p for p in Path(SOURCE_DIR).glob("**/*") if p.suffix.lower() in IMAGE_EXTS]
print(f"Found {len(images)} images in {SOURCE_DIR}")

for p in images:
    img_path = str(p)
    base = p.stem
    label_path = os.path.join(AUTO_LABEL_DIR, base + ".txt")
    if os.path.exists(label_path):
        continue
    img = cv2.imread(img_path)
    if img is None: continue
    boxes = find_display_box_by_ocr(img)
    if len(boxes) == 0:
        boxes = find_display_box_by_contour(img)
    if len(boxes) == 0:
        continue
    img_to_yolo_label(img_path, boxes, label_path)

# --------- Create dataset structure and split ---------
random.seed(SEED)
all_imgs = [p for p in Path(SOURCE_DIR).glob("**/*") if p.suffix.lower() in IMAGE_EXTS]
all_imgs = sorted([str(p) for p in all_imgs])
random.shuffle(all_imgs)

n = len(all_imgs)
if n == 0:
    raise SystemExit("No images found in SOURCE_DIR. Mount Drive or set SOURCE_DIR correctly.")

n_train = int(SPLIT['train'] * n)
n_val = int(SPLIT['val'] * n)

train_list = all_imgs[:n_train]
val_list = all_imgs[n_train:n_train+n_val]
test_list = all_imgs[n_train+n_val:]

for split_name, lst in zip(["train","val","test"],[train_list,val_list,test_list]):
    img_out_dir = os.path.join(OUTPUT_DATASET, "images", split_name)
    lbl_out_dir = os.path.join(OUTPUT_DATASET, "labels", split_name)
    os.makedirs(img_out_dir, exist_ok=True)
    os.makedirs(lbl_out_dir, exist_ok=True)
    for img_path in lst:
        fname = os.path.basename(img_path)
        base = Path(img_path).stem
        src_label = os.path.join(AUTO_LABEL_DIR, base + ".txt")
        if not os.path.exists(src_label):
            continue
        shutil.copy(img_path, os.path.join(img_out_dir, fname))
        shutil.copy(src_label, os.path.join(lbl_out_dir, base + ".txt"))

data_yaml = os.path.join(OUTPUT_DATASET, "data.yaml")
with open(data_yaml, "w") as f:
    f.write(f"path: {OUTPUT_DATASET}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write("test: images/test\n\n")
    f.write("names:\n")
    f.write(f"  0: {CLASS_NAME}\n")

# --------- Train YOLOv8 ---------
model = YOLO("yolov8n.pt")
model.train(data=data_yaml, epochs=YOLO_EPOCHS, imgsz=IMG_SIZE, batch=BATCH)

# --------- Inference on test set ---------
weights = None
runs_dir = "runs/detect"
possible = sorted(glob.glob("runs/detect/train/weights/*.pt"), key=os.path.getmtime)
if not possible:
    possible = sorted(glob.glob("runs/detect/*/weights/*.pt"), key=os.path.getmtime)
if possible:
    weights = possible[-1]

if weights is None:
    print("No weights found - aborting inference")
else:
    model = YOLO(weights)
    out_dir = "/content/inference_results"
    os.makedirs(out_dir, exist_ok=True)
    csv_rows = [("image","box_x1","box_y1","box_x2","box_y2","conf")]
    test_imgs = list(Path(OUTPUT_DATASET, "images", "test").glob("*.jpg"))
    for t in test_imgs:
        res = model.predict(source=str(t), conf=0.25, save=False)
        if len(res) > 0:
            boxes = res[0].boxes
            img = cv2.imread(str(t))
            for b in boxes:
                xyxy = b.xyxy.cpu().numpy().tolist()[0]
                conf = float(b.conf.cpu().numpy())
                x1,y1,x2,y2 = map(int, xyxy)
                cv2.rectangle(img, (x1,y1),(x2,y2),(0,255,0),2)
                cv2.putText(img, f"{conf:.2f}", (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0),2)
                csv_rows.append((str(t.name), x1,y1,x2,y2,conf))
            cv2.imwrite(os.path.join(out_dir, t.name), img)
    with open(os.path.join(out_dir, "detections.csv"), "w", newline="") as cf:
        writer = csv.writer(cf)
        writer.writerows(csv_rows)

print("Pipeline completed. Check dataset and inference_results folders.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteFound 162 images in /content
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64

In [2]:
from ultralytics import YOLO
import os
import glob

# Load best model
model = YOLO("/content/runs/detect/train/weights/best.pt")

TEST_DIR = "/content/dataset/images/test"
OUT_DIR = "/content/inference_results_fixed"

os.makedirs(OUT_DIR, exist_ok=True)

# Run inference with YOLO saving enabled
results = model.predict(
    source=TEST_DIR,
    conf=0.15,          # LOWER threshold
    iou=0.5,
    save=True,
    save_txt=True,
    project=OUT_DIR,
    name="predictions"
)

print("Inference completed.")
print("Saved to:", f"{OUT_DIR}/predictions")



image 1/17 /content/dataset/images/test/105.jpeg: 640x480 3 weighing_scales, 37.7ms
image 2/17 /content/dataset/images/test/106.jpeg: 640x384 4 weighing_scales, 41.1ms
image 3/17 /content/dataset/images/test/107.jpeg: 640x480 4 weighing_scales, 7.6ms
image 4/17 /content/dataset/images/test/12.jpeg: 640x384 5 weighing_scales, 6.5ms
image 5/17 /content/dataset/images/test/120.jpeg: 640x384 5 weighing_scales, 5.7ms
image 6/17 /content/dataset/images/test/123.jpeg: 640x384 4 weighing_scales, 6.0ms
image 7/17 /content/dataset/images/test/13.jpeg: 640x384 1 weighing_scale, 5.8ms
image 8/17 /content/dataset/images/test/145.jpeg: 640x384 5 weighing_scales, 6.2ms
image 9/17 /content/dataset/images/test/163.jpeg: 640x480 6 weighing_scales, 6.1ms
image 10/17 /content/dataset/images/test/165.jpeg: 640x384 5 weighing_scales, 6.2ms
image 11/17 /content/dataset/images/test/167.jpeg: 640x384 4 weighing_scales, 7.3ms
image 12/17 /content/dataset/images/test/17.jpeg: 640x384 4 weighing_scales, 5.7ms
im